# MAKO COMEX ingestion quickstart

This notebook mirrors `docs/INGEST_MAKO_COMEX.md` and demonstrates how to load COMEX MAKO Level-2S and Level-3 products with `alchemi`.


**Prerequisites**

Download a small flightline from the [COMEX archive](https://avirisng.jpl.nasa.gov/comex/) or point `data_root` at an existing copy. The `#!` commands below show the expected directory layout.


In [ ]:
# Download COMEX products (optional)
#! curl -O https://avirisng.jpl.nasa.gov/comex/data/mako/2014/COMEX_MAKO_20140924T221530_L2S_radiance.dat
#! curl -O https://avirisng.jpl.nasa.gov/comex/data/mako/2014/COMEX_MAKO_20140924T221530_L2S_radiance.hdr
#! curl -O https://avirisng.jpl.nasa.gov/comex/data/mako/2014/COMEX_MAKO_20140924T221530_L3_btemp.dat
#! curl -O https://avirisng.jpl.nasa.gov/comex/data/mako/2014/COMEX_MAKO_20140924T221530_L3_btemp.hdr
#! curl -O https://avirisng.jpl.nasa.gov/comex/data/mako/2014/COMEX_MAKO_20140924T221530_L3_ace.dat
#! curl -O https://avirisng.jpl.nasa.gov/comex/data/mako/2014/COMEX_MAKO_20140924T221530_L3_ace.hdr


In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import rasterio

from alchemi.ingest.mako import open_mako_ace, open_mako_btemp, open_mako_l2s
from alchemi.srf.mako import build_mako_srf_from_header
from alchemi.srf.resample import project_to_sensor


In [ ]:
def _write_envi_cube(path: Path, data: np.ndarray, wavelengths_nm: np.ndarray | None = None, *, band_names: list[str] | None = None) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    profile = {
        "driver": "ENVI",
        "height": data.shape[1],
        "width": data.shape[2],
        "count": data.shape[0],
        "dtype": "float32",
        "interleave": "bsq",
    }
    with rasterio.open(path, "w", **profile) as dst:
        dst.write(data.astype("float32"))
    header_path = path.with_suffix(".hdr")
    lines: list[str] = [
        "ENVI",
        "description = {Mock COMEX product}",
        f"samples = {profile['width']}",
        f"lines   = {profile['height']}",
        f"bands   = {profile['count']}",
        "header offset = 0",
        "file type = ENVI Standard",
        "data type = 4",
        "interleave = bsq",
        "byte order = 0",
    ]
    if wavelengths_nm is not None:
        wavelengths_um = ', '.join(f'{w / 1000.0:.6f}' for w in wavelengths_nm)
        band_mask = ', '.join('1' for _ in range(wavelengths_nm.shape[0]))
        lines.append("wavelength units = um")
        lines.append(f"wavelength = {{ {wavelengths_um} }}")
        lines.append(f"bbl = {{ {band_mask} }}")
    if band_names is not None:
        names = ', '.join(band_names)
        lines.append(f"band names = {{ {names} }}")
    header_path.write_text('\n'.join(lines) + '\n', encoding="utf-8")


def ensure_mock_comex_scene(root: Path) -> dict[str, Path]:
    root.mkdir(parents=True, exist_ok=True)
    height, width = 12, 16
    wavelengths_nm = np.linspace(7400.0, 13600.0, 128, dtype=np.float64)
    grid_y = np.linspace(-1.0, 1.0, height, dtype=np.float64)[:, None]
    grid_x = np.linspace(-1.0, 1.0, width, dtype=np.float64)[None, :]
    radial = np.sqrt(grid_x**2 + grid_y**2)

    radiance = np.stack([0.8e-2 + 0.4e-2 * np.exp(-2.5 * radial) * (1 + idx / 256.0) for idx in range(128)], axis=0)
    bt_celsius = np.stack([18.0 + 7.5 * np.exp(-radial * (1 + idx / 128.0)) for idx in range(128)], axis=0)
    ace = np.stack([np.exp(-(radial - 0.3 * band) ** 2) for band in np.linspace(0.0, 1.0, 5)], axis=0)

    l2s_dir = root / "L2S"
    l3_dir = root / "L3"
    paths = {
        "l2s": l2s_dir / "COMEX_MAKO_SAMPLE_L2S_radiance.dat",
        "bt": l3_dir / "COMEX_MAKO_SAMPLE_L3_btemp.dat",
        "ace": l3_dir / "COMEX_MAKO_SAMPLE_L3_ace.dat",
    }
    _write_envi_cube(paths['l2s'], radiance, wavelengths_nm)
    _write_envi_cube(paths['bt'], bt_celsius, wavelengths_nm)
    _write_envi_cube(paths['ace'], ace, None, band_names=['NH3', 'CO2', 'CH4', 'NO2', 'N2O'])
    return paths


In [ ]:
data_root = Path('data/mock_comex')
paths = ensure_mock_comex_scene(data_root)
paths


In [ ]:
l2s = open_mako_l2s(paths['l2s'])
print(l2s)
print('Radiance shape:', l2s['radiance'].shape)
print('Radiance units:', l2s['radiance'].attrs['units'])


In [ ]:
l3_bt = open_mako_btemp(paths['bt'])
print('BT shape:', l3_bt['bt'].shape)
print('BT units:', l3_bt['bt'].attrs['units'])


In [ ]:
l3_ace = open_mako_ace(paths['ace'])
print('ACE shape:', l3_ace['ace'].shape)
print('Gas names:', l3_ace['ace'].coords['gas_name'].values)


In [ ]:
wavelengths_nm = l2s.coords['wavelength_nm'].values
print('First five wavelengths (nm):', wavelengths_nm[:5])


In [ ]:
plt.rcParams['figure.dpi'] = 110
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(wavelengths_nm, marker='o', linestyle='-')
ax.set_xlabel('Band index')
ax.set_ylabel('Wavelength (nm)')
ax.set_title('Mako spectral grid')
fig.tight_layout()
plt.show()


In [ ]:
bt_values = l3_bt['bt'].isel(band=32).values.ravel()
fig, ax = plt.subplots(figsize=(6, 3))
ax.hist(bt_values, bins=np.linspace(bt_values.min(), bt_values.max(), 30), color='#ff7f0e')
ax.set_xlabel('Brightness temperature (K)')
ax.set_ylabel('Pixel count')
ax.set_title('Band 33 BT distribution')
fig.tight_layout()
plt.show()


In [ ]:
nh3_mean = float(l3_ace['ace'].sel(gas_name='NH3').mean())
print('NH3 ACE mean score:', round(nh3_mean, 4))


In [ ]:
mako_srf = build_mako_srf_from_header(wavelengths_nm)
highres_wavelengths = np.linspace(7400.0, 13600.0, 512)
highres_radiance = np.interp(highres_wavelengths, wavelengths_nm, l2s['radiance'].isel(y=0, x=0).values)
projected = project_to_sensor(highres_wavelengths, highres_radiance, mako_srf.centers_nm, srf=mako_srf)
print('Projected spectrum shape:', projected.shape)
